In [1]:
import re
import datetime
import pandas as pd
import numpy as np
import requests
import io
import json
import random
from main.models import *
from report.models import *
from django.db.models import Sum
from random import randint
from base64 import b64encode, b64decode
from django.contrib.auth.hashers import make_password
from django.core.files.base import ContentFile
from decimal import Decimal
from datetime import datetime, timedelta

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO

In [2]:
month=5
year=2020
sold_to_list = list(set(DailySessionllyBusinessllySale.objects.all().values_list('sold_to', flat=True)))
union_list = list(set(DailySessionllyBusinessllySale.objects.all().values_list('union', flat=True)))
    
for session in Session.objects.all():
    if MonthlySessionllyBusinessTypellySale.objects.filter(session_id=session.id, month=month, year=year).exists():
        MonthlySessionllyBusinessTypellySale.objects.filter(session_id=session.id, month=month, year=year).delete()
        print("Order deleted for date {} - {}".format(month, session.name))
    for sold_to in sold_to_list:
        print("Working on {} : {} entries".format( sold_to, session))
        for business_type in BusinessType.objects.all():
            print("\tWorking on {} business type".format(business_type.name))
            # GET ALL business type WISE SALES
            bt_sales = DailySessionllyBusinessTypellySale.objects.filter(delivery_date__month=month, delivery_date__year=year, session=session, business_type=business_type, sold_to=sold_to)
            print(bt_sales)
            # Create an User table entry for this Agent
            if bt_sales.count() > 0:
                msbts, created =  MonthlySessionllyBusinessTypellySale.objects.update_or_create(
                    month=month,
                    year=year,
                    session=session,
                    union=bt_sales[0].union,
                    business_type=business_type,
                    sold_to=sold_to,
                    defaults = {
                        'created_by':User.objects.get(username='kutobot'),
                        'modified_by':User.objects.get(username='kutobot'),
                        'total_cost':bt_sales.aggregate(Sum('total_cost'))['total_cost__sum']
                    }
                )
                if created:
                    print("\tRow Created: {}\t{}\t{}\t{}".format(msbts.month, msbts.session, msbts.union, msbts.business_type))
                else:
                    print("\tRow Exists: {}\t{}\t{}\t{}".format(msbts.month, msbts.session, msbts.union, msbts.business_type))


                # Now fill the route wise sum

                # Milk
                if getattr(MonthlySessionllyBusinessTypellySale, 'tm500_pkt', True):
                    msbts.tm500_pkt = bt_sales.aggregate(Sum('tm500_pkt'))['tm500_pkt__sum']
                    msbts.tm500_litre = bt_sales.aggregate(Sum('tm500_litre'))['tm500_litre__sum']
                    msbts.tm500_cost = bt_sales.aggregate(Sum('tm500_cost'))['tm500_cost__sum']
                    msbts.tm500_unit_price = bt_sales.aggregate(Sum('tm500_unit_price'))['tm500_unit_price__sum']


                if getattr(MonthlySessionllyBusinessTypellySale, 'std250_pkt', True):
                    msbts.std250_pkt = bt_sales.aggregate(Sum('std250_pkt'))['std250_pkt__sum']
                    msbts.std250_litre = bt_sales.aggregate(Sum('std250_litre'))['std250_litre__sum']
                    msbts.std250_cost = bt_sales.aggregate(Sum('std250_cost'))['std250_cost__sum']
                    msbts.std250_unit_price = bt_sales.aggregate(Sum('std250_unit_price'))['std250_unit_price__sum']


                if getattr(MonthlySessionllyBusinessTypellySale, 'std500_pkt', True):
                    msbts.std500_pkt = bt_sales.aggregate(Sum('std500_pkt'))['std500_pkt__sum']
                    msbts.std500_litre = bt_sales.aggregate(Sum('std500_litre'))['std500_litre__sum']
                    msbts.std500_cost = bt_sales.aggregate(Sum('std500_cost'))['std500_cost__sum']
                    msbts.std500_unit_price = bt_sales.aggregate(Sum('std500_unit_price'))['std500_unit_price__sum']


                if getattr(MonthlySessionllyBusinessTypellySale, 'fcm500_pkt', True):
                    msbts.fcm500_pkt = bt_sales.aggregate(Sum('fcm500_pkt'))['fcm500_pkt__sum']   
                    msbts.fcm500_litre = bt_sales.aggregate(Sum('fcm500_litre'))['fcm500_litre__sum']   
                    msbts.fcm500_cost = bt_sales.aggregate(Sum('fcm500_cost'))['fcm500_cost__sum']   
                    msbts.fcm500_unit_price = bt_sales.aggregate(Sum('fcm500_unit_price'))['fcm500_unit_price__sum']

                if getattr(MonthlySessionllyBusinessTypellySale, 'fcm1000_pkt', True):
                    msbts.fcm1000_pkt = bt_sales.aggregate(Sum('fcm1000_pkt'))['fcm1000_pkt__sum']   
                    msbts.fcm1000_litre = bt_sales.aggregate(Sum('fcm1000_litre'))['fcm1000_litre__sum']   
                    msbts.fcm1000_cost = bt_sales.aggregate(Sum('fcm1000_cost'))['fcm1000_cost__sum']   
                    msbts.fcm1000_unit_price = bt_sales.aggregate(Sum('fcm1000_unit_price'))['fcm1000_unit_price__sum']

                if getattr(MonthlySessionllyBusinessTypellySale, 'tmcan', True):
                    msbts.tmcan = bt_sales.aggregate(Sum('tmcan'))['tmcan__sum']
                    msbts.tmcan_litre = bt_sales.aggregate(Sum('tmcan_litre'))['tmcan_litre__sum']
                    msbts.tmcan_cost = bt_sales.aggregate(Sum('tmcan_cost'))['tmcan_cost__sum']
                    msbts.tmcan_unit_price = bt_sales.aggregate(Sum('tmcan_unit_price'))['tmcan_unit_price__sum']



                if getattr(MonthlySessionllyBusinessTypellySale, 'smcan', True):
                    msbts.smcan = bt_sales.aggregate(Sum('smcan'))['smcan__sum']
                    msbts.smcan_litre = bt_sales.aggregate(Sum('smcan_litre'))['smcan_litre__sum']
                    msbts.smcan_cost = bt_sales.aggregate(Sum('smcan_cost'))['smcan_cost__sum']
                    msbts.smcan_unit_price = bt_sales.aggregate(Sum('smcan_unit_price'))['smcan_unit_price__sum']

                if getattr(MonthlySessionllyBusinessTypellySale, 'fcmcan', True):
                    msbts.fcmcan = bt_sales.aggregate(Sum('fcmcan'))['fcmcan__sum']
                    msbts.fcmcan_litre = bt_sales.aggregate(Sum('fcmcan_litre'))['fcmcan_litre__sum']
                    msbts.fcmcan_cost = bt_sales.aggregate(Sum('fcmcan_cost'))['fcmcan_cost__sum']
                    msbts.fcmcan_unit_price = bt_sales.aggregate(Sum('fcmcan_unit_price'))['fcmcan_unit_price__sum']


                # Curd    
                if getattr(MonthlySessionllyBusinessTypellySale, 'curd500_pkt', True):
                    msbts.curd500_pkt = bt_sales.aggregate(Sum('curd500_pkt'))['curd500_pkt__sum']
                    msbts.curd500_kgs = bt_sales.aggregate(Sum('curd500_kgs'))['curd500_kgs__sum']
                    msbts.curd500_cost = bt_sales.aggregate(Sum('curd500_cost'))['curd500_cost__sum']
                    msbts.curd500_unit_price = bt_sales.aggregate(Sum('curd500_unit_price'))['curd500_unit_price__sum']


                if getattr(MonthlySessionllyBusinessTypellySale, 'curd150_pkt', True):
                    msbts.curd150_pkt = bt_sales.aggregate(Sum('curd150_pkt'))['curd150_pkt__sum']
                    msbts.curd150_kgs = bt_sales.aggregate(Sum('curd150_kgs'))['curd150_kgs__sum']
                    msbts.curd150_cost = bt_sales.aggregate(Sum('curd150_cost'))['curd150_cost__sum']
                    msbts.curd150_unit_price = bt_sales.aggregate(Sum('curd150_unit_price'))['curd150_unit_price__sum']

                if getattr(MonthlySessionllyBusinessTypellySale, 'cupcurd_box', True):
                    msbts.cupcurd_box = bt_sales.aggregate(Sum('cupcurd_box'))['cupcurd_box__sum']
                    msbts.cupcurd_count = bt_sales.aggregate(Sum('cupcurd_count'))['cupcurd_count__sum']
                    msbts.cupcurd_kgs = bt_sales.aggregate(Sum('cupcurd_kgs'))['cupcurd_kgs__sum']
                    msbts.cupcurd_cost = bt_sales.aggregate(Sum('cupcurd_cost'))['cupcurd_cost__sum']
                    msbts.cupcurd_unit_price = bt_sales.aggregate(Sum('cupcurd_unit_price'))['cupcurd_unit_price__sum']

                if getattr(MonthlySessionllyBusinessTypellySale, 'curd_bucket', True):
                    msbts.curd_bucket = bt_sales.aggregate(Sum('curd_bucket'))['curd_bucket__sum']
                    msbts.curd_bucket_kgs = bt_sales.aggregate(Sum('curd_bucket_kgs'))['curd_bucket_kgs__sum']
                    msbts.curd_bucket_cost = bt_sales.aggregate(Sum('curd_bucket_cost'))['curd_bucket_cost__sum']
                    msbts.curd_bucket_unit_price = bt_sales.aggregate(Sum('curd_bucket_unit_price'))['curd_bucket_unit_price__sum']

                if getattr(MonthlySessionllyBusinessTypellySale, 'lassi200_pkt', True):
                    msbts.lassi200_pkt = bt_sales.aggregate(Sum('lassi200_pkt'))['lassi200_pkt__sum']
                    msbts.lassi200_kgs = bt_sales.aggregate(Sum('lassi200_kgs'))['lassi200_kgs__sum']
                    msbts.lassi200_cost = bt_sales.aggregate(Sum('lassi200_cost'))['lassi200_cost__sum']
                    msbts.lassi200_unit_price = bt_sales.aggregate(Sum('lassi200_unit_price'))['lassi200_unit_price__sum']

                if getattr(MonthlySessionllyBusinessTypellySale, 'buttermilk200_pkt', True):
                    msbts.buttermilk200_pkt = bt_sales.aggregate(Sum('buttermilk200_pkt'))['buttermilk200_pkt__sum']
                    msbts.buttermilk200_litre = bt_sales.aggregate(Sum('buttermilk200_litre'))['buttermilk200_litre__sum']
                    msbts.buttermilk200_cost = bt_sales.aggregate(Sum('buttermilk200_cost'))['buttermilk200_cost__sum']
                    msbts.buttermilk200_unit_price = bt_sales.aggregate(Sum('buttermilk200_unit_price'))['buttermilk200_unit_price__sum']

                msbts.tm_litre = bt_sales.aggregate(Sum('tm_litre'))['tm_litre__sum']
                msbts.tm_cost = bt_sales.aggregate(Sum('tm_cost'))['tm_cost__sum']
                msbts.sm_litre = bt_sales.aggregate(Sum('sm_litre'))['sm_litre__sum']
                msbts.sm_cost = bt_sales.aggregate(Sum('sm_cost'))['sm_cost__sum']
                msbts.fcm_litre = bt_sales.aggregate(Sum('fcm_litre'))['fcm_litre__sum']
                msbts.fcm_cost = bt_sales.aggregate(Sum('fcm_cost'))['fcm_cost__sum']
                msbts.milk_litre = bt_sales.aggregate(Sum('milk_litre'))['milk_litre__sum']
                msbts.milk_cost = bt_sales.aggregate(Sum('milk_cost'))['milk_cost__sum']
                msbts.curd_kgs = bt_sales.aggregate(Sum('curd_kgs'))['curd_kgs__sum']
                msbts.curd_cost = bt_sales.aggregate(Sum('curd_cost'))['curd_cost__sum']
                msbts.buttermilk_cost = bt_sales.aggregate(Sum('buttermilk_cost'))['buttermilk_cost__sum']
                msbts.buttermilk_litre = bt_sales.aggregate(Sum('buttermilk_litre'))['buttermilk_litre__sum']
                msbts.lassi_litre = bt_sales.aggregate(Sum('lassi_litre'))['lassi_litre__sum']
                msbts.lassi_cost = bt_sales.aggregate(Sum('lassi_cost'))['lassi_cost__sum']
                msbts.fermented_products_litre = bt_sales.aggregate(Sum('fermented_products_litre'))['fermented_products_litre__sum']
                msbts.fermented_products_cost = bt_sales.aggregate(Sum('fermented_products_cost'))['fermented_products_cost__sum']
                msbts.total_litre = bt_sales.aggregate(Sum('total_litre'))['total_litre__sum']


                msbts.save()


Working on Agent : Eve entries
	Working on Booth business type
<QuerySet [<DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale

	Working on Other unions business type
<QuerySet []>
	Working on Other State business type
<QuerySet []>
	Working on Event / Special Order business type
<QuerySet []>
	Working on Whole Sale Dealer business type
<QuerySet [<DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySession

	Working on Own parlour business type
<QuerySet [<DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusiness

Working on Leakage : Mor entries
	Working on Booth business type
<QuerySet []>
	Working on Parlour business type
<QuerySet []>
	Working on Own parlour business type
<QuerySet []>
	Working on Private Institute business type
<QuerySet []>
	Working on Govt Institute business type
<QuerySet []>
	Working on Other unions business type
<QuerySet []>
	Working on Other State business type
<QuerySet []>
	Working on Event / Special Order business type
<QuerySet []>
	Working on Whole Sale Dealer business type
<QuerySet []>
Working on ICustomer : Mor entries
	Working on Booth business type
<QuerySet [<DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, <DailySessionllyBusinessTypellySale: DailySessionllyBusinessTypellySale object>, 